In [13]:
import numpy as np
import csv #(on doit le faire car les fichiers csv doivent avoir le même nombre de colonnes par ligne, mais les abstracts couvrent plusieurs lignes)
import re
import nltk 
import math

STOPWORDS = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at",
             "be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did",
             "didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from",
             "further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers",
             "herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just",
             "ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor",
             "not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s",
             "same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than",
             "that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through",
             "to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when",
             "where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd",
             "you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's",
             "i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've",
             "we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according",
             "accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone",
             "along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone",
             "anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth",
             "available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings",
             "begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause",
             "causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different",
             "done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough",
             "especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far",
             "ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore",
             "g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly",
             "hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however",
             "hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead",
             "invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last",
             "lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll",
             "look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile",
             "merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely",
             "nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine",
             "ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained",
             "obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall",
             "owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly",
             "possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud",
             "provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref",
             "refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results",
             "right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen",
             "self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant",
             "significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something",
             "sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still",
             "stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking",
             "tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore",
             "therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou",
             "though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards",
             "tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto",
             "upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via",
             "viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats",
             "whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither",
             "whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words",
             "world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear",
             "appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently",
             "consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly",
             "example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner",
             "insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously",
             "sure","t's","third","thorough","thoroughly","three","well","wonder"]
#Importer nos données 

data = open('train.csv')
csv_file = csv.reader(data)
data_points = []

for row in csv_file:
    data_points.append(row)
    
data_points_array = np.array(data_points)

data_without_punctuation = re.sub('\W+',' ', data_points_array[1][1])

In [14]:
#INFO: 7500 données

#on sépare les abstract et les catégories du csv

abstract_data = data_points_array[1:, 1] #

category_data = data_points_array[1:, 2]


In [15]:
#On commpte le nombre d'instance de chaque classe
"""
classe = np.unique(category_data)
print(classe)

counts = []

#On crée une liste du nombre d'occurrences des classes
for x in range(len(classe)):
    counts.append(0)

#On compte le nombre de classes dans nos données
where = []
for x in range(len(category_data)):
    where = np.where(classe ==category_data[x])
    counts[where[0][0]] += 1
print(counts)
"""

"\nclasse = np.unique(category_data)\nprint(classe)\n\ncounts = []\n\n#On crée une liste du nombre d'occurrences des classes\nfor x in range(len(classe)):\n    counts.append(0)\n\n#On compte le nombre de classes dans nos données\nwhere = []\nfor x in range(len(category_data)):\n    where = np.where(classe ==category_data[x])\n    counts[where[0][0]] += 1\nprint(counts)\n"

In [16]:
#Probabilité d'avoir une classe X dans nos données, prior P(c)
"""
prior = []

for x in counts:
    prior.append(x/len(category_data))
print(prior)
"""
def count_priors(labels):
    total = len(labels)
    unique_classes = np.unique(labels)
    count_classes = np.zeros(len(unique_classes))
    for i, unique_class in enumerate(unique_classes):
        count_classes[i] = np.count_nonzero(labels == unique_class)
    
    priors = count_classes/total
    
    priors_dict = dict(zip(unique_classes, priors))
    return priors_dict    
        

In [17]:
#Compter le nombre de mot de chaque classe


def extraction(phrase):
    extract = []
    phrase = re.sub('\W+',' ', phrase)
    phrase_array = phrase.splitlines()
    for line in phrase_array:
        word_array = line.split(' ')
        for m in word_array:
            m = m.lower().strip()
            if m.lower() not in STOPWORDS and m != '':
                extract.append(m)
            
    return extract #retourne une list de mots

def get_vocab(corpus):
    entire = []
    for abstract in corpus:
        words = extraction(abstract)
        entire = entire + words
        
    vocabulaire = np.unique(entire)
    return vocabulaire


#Nombre d'occurences total pour tous les mots selon une classe
def sac_de_mot_general(sac, labels ,classe, vocabulary_bag):
    size_vocab = len(vocabulary_bag)
    mots_classe = []
    bag = dict()
    for i in range(len(sac)):
        if classe == labels[i]:
            mots = extraction(sac[i])
            for mot in mots:
                mots_classe.append(mot)
                if mot not in bag.keys():
                    bag[mot] = 1
                else:
                    bag[mot] += 1
    
    vocabulary_classe = np.unique(mots_classe)
    
    total = size_vocab #on ajoute la taille du vocabulaire
    
    words_not_in_class = np.setdiff1d(vocabulary_bag, vocabulary_classe)
    
    for i in bag:
        total = total+ bag[i]
    
    
    for i in bag:
        bag[i] = bag[i]/total
    
    for i in words_not_in_class:
        bag[i] = 1/total
    
    return bag


def sac_de_mot_final(sac, labels):
    dictio_final = {}
    classe = np.unique(labels)
    vocabulary_bag = get_vocab(sac)
    for i in classe:
        dictio_final[i] = sac_de_mot_general(sac, labels ,i, vocabulary_bag)
        
    return dictio_final
        
#Nombre d'occurences total pour un article
def sac_de_mot(sac):
    bag = dict()
    mots = extraction(sac)
    for mot in mots:
        if mot not in bag.keys():
            bag[mot] = 1
        else:
            bag[mot] += 1
    return bag

#Probabillité d'avoir un mot sachant une classe, vraisemblance P(c/x)

In [18]:
class ClassifieurBayesienNaif:
    def train(self, data, labels):
        self.data = data
        self.labels = labels
        self.probabilities = sac_de_mot_final(data, labels)
        self.priors = count_priors(labels)
        
    def classify(self, test_input):
        shape_of_test = test_input.shape
        size = shape_of_test[0]
        pred = []
        if size == 1:
            input_dictio = sac_de_mot(test_input)
            likelihood_class = {}
            for bag_proba in self.probabilities:
                prior = self.priors[bag_proba]
                probability = prior  #on commence avec la valeur du prior vu que l'on va le multipier au produit
                for key in input_dictio:
                    number_of_occurence = input_dictio[key]
                    if key in self.probabilities[bag_proba]:
                        proba_word_given_class = self.probabilities[bag_proba][key]
                        proba = proba_word_given_class ** number_of_occurence
                    else:
                        proba = 1
                    probability = probability * proba
                    
                likelihood_class[bag_proba] = probability
                
            values_prob = list(likelihood_class.values())
            keys_prob = list(likelihood_class.keys())
            best_probability = max(values_prob)
            class_predicted = keys_prob[values_prob.index(best_probability)]
            pred.append(class_predicted)
                
                    
        else:
            for i, input_data_row in enumerate(test_input):
                input_dictio = sac_de_mot(input_data_row)
                likelihood_class = {}
                for bag_proba in self.probabilities:
                    prior = self.priors[bag_proba]
                    probability = math.log(prior) #on commence avec la valeur du prior vu que l'on va le multipier au produit
                    for key in input_dictio:
                        number_of_occurence = input_dictio[key]
                        if key in self.probabilities[bag_proba]:
                            proba_word_given_class = self.probabilities[bag_proba][key]
                            proba = proba_word_given_class ** number_of_occurence
                            log_proba = math.log(proba)
                            
                        else:
                            proba = 1
                            log_proba = 0
                        probability = probability + log_proba
                        
                    likelihood_class[bag_proba] = probability
                    
                values_prob = list(likelihood_class.values())
                keys_prob = list(likelihood_class.keys())
                best_probability = max(values_prob)
                class_predicted = keys_prob[values_prob.index(best_probability)]
                pred.append(class_predicted)
                
        return pred
                
        
        
        

In [19]:
test = open('test.csv')
csv_file_test = csv.reader(test)
test_points = []

for row in csv_file_test:
    test_points.append(row)
    
test_points_array = np.array(test_points)

test_abstract_data = test_points_array[1:, 1]


In [20]:
classifier = ClassifieurBayesienNaif()
classifier.train(abstract_data,category_data)
classes = classifier.classify(test_abstract_data)
classes_array = np.array(classes)
indices = np.arange(15000)
final = np.column_stack((indices,classes_array))
header = ["Id", "Category"]
to_csv = np.vstack([header, final])

In [21]:
print(to_csv[0:10])

[['Id' 'Category']
 ['0' 'stat.ML']
 ['1' 'astro-ph.SR']
 ['2' 'astro-ph.SR']
 ['3' 'math.AP']
 ['4' 'cs.LG']
 ['5' 'gr-qc']
 ['6' 'math.CO']
 ['7' 'cond-mat.mtrl-sci']
 ['8' 'astro-ph.CO']]


In [22]:
np.savetxt('submission.csv', to_csv, delimiter=',', fmt='%s')